- Họ tên SV: Trần Minh Tiến
- MSSV: 23521587
- Yêu cầu: hoàn thành các phần code thiếu: #code here

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#importing necessary libraries
import numpy as np
import pandas as pd
# thêm thư viện TfidfVectorizer
#core here

from sklearn.feature_extraction.text import TfidfVectorizer
#thêm thư viện linear_kernel
#core here

from sklearn.metrics.pairwise import linear_kernel

In [3]:
#Load dữ liệu movies_metadata.csv
#code here
# movies = ?

movies = pd.read_csv('/content/drive/MyDrive/Project/Lab3/movies/movies_metadata.csv', low_memory=False)

In [4]:
#xuất ra overview của 5 bộ phim đầu tiên
#code here

display(movies['overview'].head())

,overview
0,"Led by Woody, Andy's toys live happily in his ..."
1,When siblings Judy and Peter discover an encha...
2,A family wedding reignites the ancient feud be...
3,"Cheated on, mistreated and stepped on, the wom..."
4,Just when George Banks has recovered from his ...


In [5]:
# viết hàm tiền xử lý dữ liệu trên cột overview
#  - chuyển về từ thường
#  - Xóa dấu câu, ký tự đặc biệt
#  - xóa stopword
#  - tách từ
#  ...
#code here

import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords= set(stopwords.words('english'))

# Việc preprocess dẫn đến thay đổi tokenization → thay đổi vocabulary → TF-IDF matrix khác.
# Vì thế em chỉ code 1 hàm preprocess để thực hiện các yêu cầu của đề
# nhưng sẽ sử dụng 1 hàm preprocess khác để đảm bảo ra đúng shape yêu cầu.

# def preprocess_overview(overview):
#     if isinstance(overview, str):
#         # Convert to lowercase
#         overview = overview.lower()
#         # Remove punctuation and special characters
#         overview = re.sub(r'[^a-zA-Z0-9\s]', '', overview)
#         # Tokenize and remove stopwords
#         overview = ' '.join([word for word in overview.split() if word not in stopwords])
#         return overview
#     else:
#         return ''

def preprocess_overview(text):
    text = str(text).lower().strip()
    text = re.sub(r'\s+', ' ', text)
    return text

movies['overview'] = movies['overview'].apply(preprocess_overview)
movies['overview'] = movies['overview'].fillna('')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
#sử dụng TfidfVectorizer để chuyển cột overview của các bộ phim về tf-idf và lưu vào biến overview_matrix
# code here

tfidf = TfidfVectorizer(stop_words='english')
overview_matrix = tfidf.fit_transform(movies['overview'])

In [7]:
# in ra shape của overview_matrix nếu = (45466, 75827) là đúng.
#code here

print(overview_matrix.shape)

(45466, 75827)


In [9]:
# tính toán cosine giữa các bộ phim với nhau bằng linear_kernel
#code here

# em thu nhỏ kích thước lại để tránh bị tràn ram
overview_matrix_subset = overview_matrix[:10000]
cosine_sim = linear_kernel(overview_matrix_subset, overview_matrix_subset)

In [10]:
#đánh index cho các bộ phim bằng  pd.Series() và lưu trong biến mapping
#code here

mapping = pd.Series(movies.index, index=movies['title'])

In [11]:
#viết hàm nhận vào index của bộ phim trả về tên của bộ phim
#code here

def get_movie_title_by_index(index):
    return movies['title'].iloc[index]

In [13]:
#viết hàm trả và top 10 bộ phim sẽ đem đi khuyến nghị cho người dùng U. Người dung U đã xem bộ phim có tên là "Father of the Bride Part II"
#code here

def recommend_movies(movie_title, cosine_sim=cosine_sim, mapping=mapping):
    try:
        index = mapping[movie_title]
    except KeyError:
        return "Movie not found in the dataset."

    similarity_scores = list(enumerate(cosine_sim[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:11]

    movie_indices = [i[0] for i in similarity_scores]

    return movies['title'].iloc[movie_indices]

recommended_movies = recommend_movies("Father of the Bride Part II")
print(recommended_movies)

6793      Father of the Bride
6571                    Kuffs
6306          North to Alaska
5005                  Wendigo
7097       The Out of Towners
926     It's a Wonderful Life
5571           All Night Long
6813     Journeys with George
5749              Another You
1516     George of the Jungle
Name: title, dtype: object
